In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from kerastuner import HyperModel

In [ ]:
class GeneratorHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        # Encoder
        input_layer = layers.Input(shape=self.input_shape)
        x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
        x = layers.BatchNormalization()(x)
        
        filters = hp.Int('filters', min_value=64, max_value=256, step=32)
        x = layers.Conv2D(filters, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)

        # Transformer
        num_residual_blocks = hp.Int('num_residual_blocks', min_value=1, max_value=10, step=1)
        for _ in range(num_residual_blocks):
            x = residual_block(x, filters=filters)

        # Decoder
        x = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        output_layer = layers.Conv2D(3, (3, 3), activation='tanh', padding='same')(x)

        generator_model = Model(input_layer, output_layer)

        return generator_model


In [ ]:
class DiscriminatorHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        input_layer = layers.Input(shape=self.input_shape)
        
        filters = hp.Int('filters', min_value=32, max_value=128, step=32)
        
        x = layers.Conv2D(filters, (3, 3), strides=(2, 2), activation='relu', padding='same')(input_layer)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(filters * 2, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(filters * 4, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Flatten()(x)
        output_layer = layers.Dense(1, activation='sigmoid')(x)

        discriminator_model = Model(input_layer, output_layer)

        return discriminator_model
